In [1]:

import os
import datetime as dt
import pandas as pd
from fyers_apiv3 import fyersModel
import pytz
import time


#generate trading session
client_id = open("../client_ID.txt",'r').read()
access_token = open("../access_token.txt",'r').read()

# Initialize the FyersModel instance with your client_id, access_token, and enable async mode
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="D:\FyiersApiAutomation\logs")

def fetchOHLC_full(ticker,interval,inception_date):

    from_date = dt.datetime.strptime(inception_date, '%Y-%m-%d')
    to_date = dt.date.today()

    # Create a DataFrame
    columns = ['Timestamp','Open','High','Low','Close','Volume']
    df = pd.DataFrame(columns=columns)

    while True:
        from_date_string = from_date.strftime("%Y-%m-%d")
        ttoday = dt.date.today()
        to_date_string = ttoday.strftime("%Y-%m-%d")

        if from_date.date() >= (dt.date.today() - dt.timedelta(50)):
            data = {
                "symbol":ticker,
                "resolution":interval,
                "date_format":"1",
                "range_from":from_date_string,
                "range_to":to_date_string,
                "cont_flag":"1"
            }
            resp = fyers.history(data=data)['candles']
            df1 = pd.DataFrame(resp, columns=columns)
            result = pd.concat([df, df1], ignore_index=True)
            df = result
            break
        else:
            to_date = from_date + dt.timedelta(50)
            to_date_string = to_date.strftime("%Y-%m-%d")
            data = {
                "symbol":ticker,
                "resolution":interval,
                "date_format":"1",
                "range_from":from_date_string,
                "range_to":to_date_string,
                "cont_flag":"1"
            }
            resp = fyers.history(data=data)['candles']
            df1 = pd.DataFrame(resp, columns=columns)
            result = pd.concat([df, df1], ignore_index=True)
            df = result
            from_date = to_date + dt.timedelta(1)

    # Convert Timestamp to datetime in UTC
    df['Date'] = pd.to_datetime(df['Timestamp'],unit='s').dt.tz_localize(pytz.utc)

    # Convert Timestamp to IST
    ist = pytz.timezone('Asia/Kolkata')
    df['Date'] = df['Date'].dt.tz_convert(ist)
    df = df.drop(columns=['Timestamp'])

    time.sleep(10)

    return df[['Date', 'Open', 'High', 'Low', 'Close']]
    #return (df)

# Fetch OHLC data using the function
response_df = fetchOHLC_full("NSE:SBIN-EQ","D","2025-08-20")

# # Print the DataFrame
print(response_df)




                       Date    Open    High     Low   Close
0 2025-08-20 05:30:00+05:30  828.70  831.90  827.40  828.95
1 2025-08-21 05:30:00+05:30  828.95  833.25  824.05  825.70
2 2025-08-22 05:30:00+05:30  824.00  826.00  815.70  816.25


In [3]:
def calculate_ema(data, period=200, column="Close"):
    data[f"EMA_{period}"] = data[column].ewm(span=period, adjust=False).mean()
    return data

In [4]:
def generate_signals(data, ema_column="EMA_200"):
    """
    Generate Buy signals when candle intercepts EMA_200.
    """
    signals = []
    for i in range(1, len(data)):
        prev_close = data["Close"].iloc[i-1]
        prev_ema   = data[ema_column].iloc[i-1]
        curr_close = data["Close"].iloc[i]
        curr_ema   = data[ema_column].iloc[i]

        if prev_close < prev_ema and curr_close > curr_ema:
            signals.append((data.index[i], "BUY", curr_close))
    return signals


In [5]:
def calculate_atr(data, period=14):
    data["H-L"] = data["High"] - data["Low"]
    data["H-C"] = abs(data["High"] - data["Close"].shift(1))
    data["L-C"] = abs(data["Low"] - data["Close"].shift(1))
    data["TR"] = data[["H-L", "H-C", "L-C"]].max(axis=1)
    data["ATR"] = data["TR"].rolling(window=period).mean()
    return data

In [6]:
def create_trades(data, signals):
    trades = []
    for date, signal, entry in signals:
        if signal == "BUY":
            low_price = data.loc[date, "Low"]
            atr_value = data.loc[date, "ATR"]
            stoploss = low_price - atr_value
            risk = entry - stoploss
            target = entry + (2 * risk)
            trades.append({
                "Entry_Date": date,
                "Entry": round(entry,2),
                "Stoploss": round(stoploss,2),
                "Target": round(target,2),
                "Status": "OPEN"
            })
    return trades

In [7]:
def simulate_exits(data, trades):
    results = []
    for trade in trades:
        entry_date = trade["Entry_Date"]
        entry_price = trade["Entry"]
        stoploss = trade["Stoploss"]
        target = trade["Target"]

        # Look ahead from entry date
        entry_idx = data.index.get_loc(entry_date)
        future_data = data.iloc[entry_idx+1:]  

        exit_price = None
        exit_date = None
        status = None
        pnl = 0

        for date, row in future_data.iterrows():
            low = row["Low"]
            high = row["High"]

            # Check Stoploss first
            if low <= stoploss:
                exit_price = stoploss
                exit_date = date
                status = "LOSS"
                pnl = exit_price - entry_price
                break

            # Check Target
            if high >= target:
                exit_price = target
                exit_date = date
                status = "WIN"
                pnl = exit_price - entry_price
                break

        # If no exit found till end of data
        if status is None:
            exit_price = data.iloc[-1]["Close"]
            exit_date = data.index[-1]
            status = "OPEN"
            pnl = exit_price - entry_price

        trade.update({
            "Exit_Date": exit_date,
            "Exit_Price": round(exit_price,2),
            "Result": status,
            "PnL": round(pnl,2)
        })
        results.append(trade)

    return results

In [12]:
def test_strategy(symbols):
    all_results = {}
    for symbol in symbols:
        df = fetchOHLC_full(symbol,"D","2021-01-01")
        output_csv = df.to_csv("EMA_Breakout_OHLC.csv")
        df = calculate_ema(df, 200)
        df = calculate_atr(df)

        signals = generate_signals(df)
        signals_out = pd.DataFrame(signals)
        output_signals = signals_out.to_csv("EMA_Breakout_signals.csv")
        trades = create_trades(df, signals)
        trades_out = pd.DataFrame(trades)
        output_trades = trades_out.to_csv("EMA_Breakout_trades.csv")
        results = simulate_exits(df, trades)
        results_out = pd.DataFrame(results)
        output_results = results_out.to_csv("EMA_Breakout_results.csv")

        all_results[symbol] = results
    return all_results


In [13]:
symbols = ["NSE:SBIN-EQ"]
strategy_results = test_strategy(symbols)

for sym, trades in strategy_results.items():
    print(f"\n--- {sym} ---")
    for t in trades[:3]:   # Show first 3 trades per symbol
        print(t)


--- NSE:SBIN-EQ ---
{'Entry_Date': 18, 'Entry': 282.35, 'Stoploss': 258.08, 'Target': 330.89, 'Status': 'OPEN', 'Exit_Date': 21, 'Exit_Price': 330.89, 'Result': 'WIN', 'PnL': 48.54}
{'Entry_Date': 294, 'Entry': 468.7, 'Stoploss': 444.16, 'Target': 517.77, 'Status': 'OPEN', 'Exit_Date': 313, 'Exit_Price': 517.77, 'Result': 'WIN', 'PnL': 49.07}
{'Entry_Date': 347, 'Entry': 474.6, 'Stoploss': 455.7, 'Target': 512.4, 'Status': 'OPEN', 'Exit_Date': 357, 'Exit_Price': 455.7, 'Result': 'LOSS', 'PnL': -18.9}
